In [4]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 44.1 MB/s eta 0:00:00


# ***FACEMESH***

In [3]:
import cv2
import mediapipe as mp
import time
from IPython.display import Video

class FaceMeshDetector:
    def __init__(self, staticMode=False, maxFaces=2, minDetectionCon=0.5, minTrackCon=0.5):
        self.staticMode = staticMode
        self.maxFaces = maxFaces
        self.minDetectionCon = minDetectionCon
        self.minTrackCon = minTrackCon
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceMesh = mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(
            static_image_mode=self.staticMode,
            max_num_faces=self.maxFaces,
            min_detection_confidence=self.minDetectionCon,
            min_tracking_confidence=self.minTrackCon
        )
        self.drawSpec = self.mpDraw.DrawingSpec(thickness=1, circle_radius=2)

    def findFaceMesh(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.faceMesh.process(imgRGB)
        faces = []
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(
                        img, faceLms, mp.solutions.face_mesh.FACEMESH_CONTOURS,
                        self.drawSpec, self.drawSpec)
                face = []
                for lm in faceLms.landmark:
                    ih, iw, ic = img.shape
                    x, y = int(lm.x * iw), int(lm.y * ih)
                    face.append([x, y])
                faces.append(face)
        return img, faces

def main():
    # Load video file
    cap = cv2.VideoCapture("/content/drive/MyDrive/WIN_20241025_12_47_03_Pro.mp4")
    pTime = 0
    detector = FaceMeshDetector(maxFaces=2)

    # Define video writer for output
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter('output_with_mesh.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    while cap.isOpened():
        success, img = cap.read()
        if not success:
            break

        img, faces = detector.findFaceMesh(img)
        if faces:
            print(faces[0])  # Print the first detected face landmarks for reference

        # Calculate FPS
        cTime = time.time()
        fps = 1 / (cTime - pTime) if (cTime - pTime) != 0 else 0
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,
                    3, (0, 255, 0), 3)

        # Write each frame to the output video
        out.write(img)

    cap.release()
    out.release()
    print("Video processing complete. Saved as output_with_mesh.mp4")

# Run main to process the video
main()

# Display the output video
Video("output_with_mediapipe_mesh.mp4", embed=True)



ModuleNotFoundError: No module named 'mediapipe'

# ***FACELANDMARK***

In [ ]:
from google.colab import drive
import cv2
import mediapipe as mp
import time
from IPython.display import Video, display

# Mount Google Drive
drive.mount('/content/drive')

class FaceMeshDetector:
    def __init__(self, staticMode=False, maxFaces=2, minDetectionCon=0.5, minTrackCon=0.5):
        self.staticMode = staticMode
        self.maxFaces = maxFaces
        self.minDetectionCon = minDetectionCon
        self.minTrackCon = minTrackCon
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceMesh = mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(
            static_image_mode=self.staticMode,
            max_num_faces=self.maxFaces,
            min_detection_confidence=self.minDetectionCon,
            min_tracking_confidence=self.minTrackCon
        )
        self.drawSpec = self.mpDraw.DrawingSpec(thickness=1, circle_radius=2)

    def findFaceMesh(self, img, draw=True, highlight_count=0):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.faceMesh.process(imgRGB)
        faces = []
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                face = []
                for id, lm in enumerate(faceLms.landmark):  # Use all landmarks
                    ih, iw, ic = img.shape
                    x, y = int(lm.x * iw), int(lm.y * ih)
                    face.append([x, y])

                    if draw:

                        if id < highlight_count:
                            cv2.circle(img, (x, y), 4, (0, 0, 255), -1)  # Highlight color for specified landmarks
                        else:
                            cv2.circle(img, (x, y), 2, (0, 255, 0), -1)  # Regular color for all landmarks
                faces.append(face)
        return img, faces

def main():
    # Ask the user for the number of landmarks to highlight
    try:
        highlight_count = int(input("Enter the number of landmarks to highlight : "))
        if highlight_count > 468:  # Update according to MediaPipe Face Mesh total landmarks
            raise ValueError("Number of landmarks cannot exceed 468.")
    except ValueError:
        print("Invalid input. Using default value of 68 landmarks.")
        highlight_count = 68  # Default value if input is invalid

    # Load your video file
    cap = cv2.VideoCapture("/content/drive/MyDrive/WIN_20241029_12_25_48_Pro (1).mp4")  # Replace with the correct filename
    pTime = 0
    detector = FaceMeshDetector(maxFaces=2)

    # Define video writer for output - specify Google Drive path
    output_path = '/content/drive/MyDrive/output_with_Highlighted_landmark.mp4'  # Change to your desired location in Google Drive
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    while cap.isOpened():
        success, img = cap.read()
        if not success:
            break

        # Process each frame with the specified number of landmarks
        img, faces = detector.findFaceMesh(img, highlight_count=highlight_count)
        if faces:
            print(faces[0])  # Print the first detected face landmarks for reference

        # Calculate FPS
        cTime = time.time()
        fps = 1 / (cTime - pTime) if (cTime - pTime) != 0 else 0
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,
                    3, (0, 255, 0), 3)

        # Write each frame to the output video
        out.write(img)

    cap.release()
    out.release()
    print(f"Video processing complete. Saved to Google Drive as {output_path}")

# Run main to process the video
main()

# Display the output video from Google Drive
display(Video('/content/drive/MyDrive/output_with_Highlighted_landmark.mp4', embed=True))


Output hidden; open in https://colab.research.google.com to view.

# ***FACELANDMESH_WITH_GIVEN_LANDMARK_POINTS***

In [ ]:
import cv2
import mediapipe as mp
import time
from IPython.display import Video
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

class FaceMeshDetector:
    def __init__(self, staticMode=False, maxFaces=2, minDetectionCon=0.5, minTrackCon=0.5):
        self.staticMode = staticMode
        self.maxFaces = maxFaces
        self.minDetectionCon = minDetectionCon
        self.minTrackCon = minTrackCon
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceMesh = mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(
            static_image_mode=self.staticMode,
            max_num_faces=self.maxFaces,
            min_detection_confidence=self.minDetectionCon,
            min_tracking_confidence=self.minTrackCon
        )
        self.drawSpec = self.mpDraw.DrawingSpec(thickness=1, circle_radius=2)

    def findFaceMesh(self, img, highlight_count=0):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.faceMesh.process(imgRGB)
        faces = []
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                # Draw the complete mesh (optional)
                self.mpDraw.draw_landmarks(
                    img, faceLms, mp.solutions.face_mesh.FACEMESH_CONTOURS,
                    self.drawSpec, self.drawSpec)

                face = []
                for id, lm in enumerate(faceLms.landmark):
                    ih, iw, ic = img.shape
                    x, y = int(lm.x * iw), int(lm.y * ih)
                    face.append([x, y])

                    # Highlight only specified landmarks
                    if id < highlight_count:
                        cv2.circle(img, (x, y), 4, (0, 0, 255), -1)  # Highlight color (red)
                faces.append(face)
        return img, faces

def main():
    # Get user input for number of landmarks to highlight
    try:
        highlight_count = int(input("Enter the number of landmarks to highlight : "))
        highlight_count = min(highlight_count, 468)  # Ensure it does not exceed 468
    except ValueError:
        print("Invalid input. Using default value of 68 landmarks.")
        highlight_count = 68  # Default value if input is invalid

    # Load your video file
    cap = cv2.VideoCapture("/content/drive/MyDrive/WIN_20241029_12_46_11_Pro.mp4")
    pTime = 0
    detector = FaceMeshDetector(maxFaces=2)

    # Define video writer for output
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    output_path = '/content/drive/MyDrive/output_with_given_landmarks.mp4'  # Specify the output path in Google Drive
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    while cap.isOpened():
        success, img = cap.read()
        if not success:
            break

        img, faces = detector.findFaceMesh(img, highlight_count)
        if faces:
            print(faces[0])  # Print the first detected face landmarks for reference

        # Calculate FPS
        cTime = time.time()
        fps = 1 / (cTime - pTime) if (cTime - pTime) != 0 else 0
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,
                    3, (0, 255, 0), 3)

        # Write each frame to the output video
        out.write(img)

    cap.release()
    out.release()
    print(f"Video processing complete. Saved as {output_path}")

# Run main to process the video
main()

# Display the output video
Video("/content/drive/MyDrive/output_with_given_landmarks.mp4", embed=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter the number of landmarks to highlight : 68
[[814, 723], [805, 702], [814, 709], [805, 680], [804, 695], [806, 686], [813, 663], [784, 665], [814, 646], [813, 636], [812, 593], [815, 727], [816, 730], [818, 731], [818, 736], [817, 739], [817, 743], [818, 747], [821, 756], [808, 706], [806, 706], [774, 629], [799, 667], [794, 669], [789, 669], [783, 666], [802, 666], [789, 651], [794, 651], [784, 652], [781, 655], [780, 669], [806, 768], [782, 663], [775, 662], [777, 663], [791, 696], [807, 722], [810, 730], [801, 725], [799, 727], [806, 730], [803, 731], [798, 740], [802, 702], [800, 696], [775, 647], [801, 678], [796, 700], [797, 696], [777, 695], [802, 687], [783, 641], [777, 643], [775, 617], [803, 647], [799, 653], [793, 732], [790, 732], [801, 704], [805, 707], [798, 732], [801, 732], [775, 637], [798, 703], [791, 642], [789, 636], [785, 600], [774, 

# ***FACELANDMARK WITH GIVEN POINTS***

In [8]:
import cv2
import mediapipe as mp
import time
from IPython.display import Video
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

class FaceMeshDetector:
    def __init__(self, staticMode=False, maxFaces=4, minDetectionCon=0.5, minTrackCon=0.5):
        self.staticMode = staticMode
        self.maxFaces = maxFaces
        self.minDetectionCon = minDetectionCon
        self.minTrackCon = minTrackCon
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceMesh = mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(
            static_image_mode=self.staticMode,
            max_num_faces=self.maxFaces,
            min_detection_confidence=self.minDetectionCon,
            min_tracking_confidence=self.minTrackCon
        )
        self.drawSpec = self.mpDraw.DrawingSpec(thickness=1, circle_radius=2)

    def findFaceMesh(self, img, highlight_count=0):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.faceMesh.process(imgRGB)
        faces = []
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                face = []
                for id, lm in enumerate(faceLms.landmark):
                    ih, iw, ic = img.shape
                    x, y = int(lm.x * iw), int(lm.y * ih)
                    face.append([x, y])

                    # Highlight only specified landmarks
                    if id < highlight_count:
                        cv2.circle(img, (x, y), 4, (0, 0, 255), -1)  # Highlight color (red)
                faces.append(face)
        return img, faces

def main():
    # Get user input for number of landmarks to highlight
    try:
        highlight_count = int(input("Enter the number of landmarks to highlight : "))
        highlight_count = min(highlight_count, 468)  # Ensure it does not exceed 468
    except ValueError:
        print("Invalid input. Using default value of 68 landmarks.")
        highlight_count = 68  # Default value if input is invalid

    # Load your video file
    cap = cv2.VideoCapture("/content/drive/MyDrive/CV.mp4")  # Replace with the correct filename
    pTime = 0
    detector = FaceMeshDetector(maxFaces=2)

    # Define video writer for output
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    output_path = '/content/drive/MyDrive/output_with_given_points_landmarks.mp4'  # Specify the output path in Google Drive
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    while cap.isOpened():
        success, img = cap.read()
        if not success:
            break

        img, faces = detector.findFaceMesh(img, highlight_count)
        if faces:
            print(faces[0])  # Print the first detected face landmarks for reference

        # Calculate FPS
        cTime = time.time()
        fps = 1 / (cTime - pTime) if (cTime - pTime) != 0 else 0
        pTime = cTime
        cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,
                    3, (0, 255, 0), 3)

        # Write each frame to the output video
        out.write(img)

    cap.release()
    out.release()
    print(f"Video processing complete. Saved as {output_path}")

# Run main to process the video
main()

# Display the output video
Video("/content/drive/MyDrive/output_with_given_points_landmarks.mp4", embed=True)


Output hidden; open in https://colab.research.google.com to view.